In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/samsum-dataset/samsum-train.csv
/kaggle/input/samsum-dataset/samsum-test.csv
/kaggle/input/samsum-dataset/samsum-validation.csv


In [2]:
# Import required libraries
import pandas as pd
import numpy as np
# from datasets import load_dataset
from datasets import Dataset
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
import random


2024-07-24 04:08:30.520975: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 04:08:30.521077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 04:08:30.653153: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# # Load the train dataset (CNN dialy mail)
# ds = load_dataset("abisee/cnn_dailymail", "3.0.0")
# print(ds)


# using samsum dataset due to less computational power
train_df = pd.read_csv("/kaggle/input/samsum-dataset/samsum-train.csv")
test_df = pd.read_csv("/kaggle/input/samsum-dataset/samsum-test.csv")
val_df = pd.read_csv("/kaggle/input/samsum-dataset/samsum-validation.csv")

# converting to pandas dataframe
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [4]:
# Load tokenizer and model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [5]:
# Define preprocess function
def preprocess_function(examples):
    # Ensure the columns are strings
    inputs = [str(text) for text in examples["dialogue"]]
    targets = [str(text) for text in examples["summary"]]
    
    model_inputs = tokenizer(inputs, max_length=1024, padding="max_length", truncation=True)
    labels = tokenizer(targets, max_length=1024, padding="max_length", truncation=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [7]:
from transformers import BartTokenizer, BartForConditionalGeneration, DataCollatorForSeq2Seq, Trainer, TrainingArguments

# Initialize data collator for seq2seq tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./text_summarization_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    load_best_model_at_end=True,
    fp16=True,  # Enable mixed precision
    gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)

# Train the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
0,1.164200,0.042581
1,0.045200,0.041168
2,0.041900,0.040805


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in 

TrainOutput(global_step=2760, training_loss=0.24741179597550544, metrics={'train_runtime': 9880.3116, 'train_samples_per_second': 4.473, 'train_steps_per_second': 0.279, 'total_flos': 2.69259619958784e+16, 'train_loss': 0.24741179597550544, 'epoch': 2.997556339940266})

In [8]:
# Evaluate the model
eval_results = trainer.evaluate(eval_dataset=tokenized_test)
print(eval_results)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.040996383875608444, 'eval_runtime': 65.8341, 'eval_samples_per_second': 12.44, 'eval_steps_per_second': 3.114, 'epoch': 2.997556339940266}


In [9]:
# Save the model and tokenizer to output directory
output_dir = './text_summarization_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./text_summarization_model/tokenizer_config.json',
 './text_summarization_model/special_tokens_map.json',
 './text_summarization_model/vocab.json',
 './text_summarization_model/merges.txt',
 './text_summarization_model/added_tokens.json')

In [11]:

# Zip the saved model directory
import shutil
shutil.make_archive("/kaggle/working/text_summarization_model/checkpoint-2760", 'zip', "/kaggle/working/text_summarization_model/checkpoint-2760")

'/kaggle/working/text_summarization_model/checkpoint-2760.zip'